In [ ]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import numpy as np

"""
Update Parameters Here
"""
COLLECTION_NAME = "quaks"
CONTRACT = '0x07bbdaf30e89ea3ecf6cadc80d6e7c4b0843c729'

# This doesnt usually change
METHOD = "raritytools"

# Need to updte these depending on the collection
START_TIME = datetime.datetime.strptime("2021-07-27T21:20:47", "%Y-%m-%dT%H:%M:%S")
LAST_MINT = datetime.datetime.strptime("2021-08-02T06:05:00", "%Y-%m-%dT%H:%M:%S")
TIME_DELTA = 0.5 # length of a single mint window

In [ ]:
"""
Generate Dataset
"""
RARITY_CSV = "../metadata/rarity_data/{}_{}.csv".format(COLLECTION_NAME, METHOD)
RARITY_DB = pd.read_csv(RARITY_CSV)
MINT_DB = pd.read_csv("../minting_data/{}_minting.csv".format(COLLECTION_NAME))

MINT_DB['time'] =  pd.to_datetime(MINT_DB['time'], format="%Y-%m-%dT%H:%M:%S")
MINT_DB = MINT_DB.sort_values(by=['rank'])


MINT_DB['before_time'] = MINT_DB['time'] - datetime.timedelta(minutes = TIME_DELTA)
MINT_DB['after_time'] = MINT_DB['time'] + datetime.timedelta(minutes = TIME_DELTA)

after_time = MINT_DB['after_time']
before_time = MINT_DB['before_time']
time = MINT_DB['time']

minted_in_epoch = []

for i in range(0, len(before_time)):
    sliding_window = MINT_DB[(MINT_DB['time'] > before_time[i]) & (MINT_DB['time'] < time[i])]
    minted_in_epoch.append(len(sliding_window))

MINT_DB['minted'] = minted_in_epoch

MINT_DB.to_csv("output.csv")

In [ ]:
"""
Generate Plot
"""
fig, ax = plt.subplots(1,1, figsize=(14, 7))
plt.bar(MINT_DB['rank'], MINT_DB['minted'], width=0.3)
plt.xlabel('rank', fontsize=12)
plt.ylabel('minted in window', fontsize=12)
ax.set_xticks(ax.get_xticks()[::2])
plt.xlim(0, 10000)